# Structured Output으로 정형화된 응답 받기

이 노트북에서는 **Structured Output**을 사용하여 LLM 응답을 정해진 스키마(구조)에 맞게 받는 방법을 알아봅니다.

## Structured Output이란?

LLM의 응답을 **Pydantic 모델**로 정의한 구조에 맞게 강제하는 기능입니다.

### 일반 응답 vs Structured Output

| 구분 | 일반 응답 | Structured Output |
|------|----------|------------------|
| 반환 타입 | 문자열 (str) | Pydantic 객체 |
| 형식 | 자유 형식 텍스트 | 정의된 필드 구조 |
| 파싱 | 수동 파싱 필요 | 자동 파싱 |
| 안정성 | 형식 불일치 가능 | 스키마 보장 |

## 언제 사용할까?

1. **API 응답 생성**: JSON 형식으로 프론트엔드에 전달
2. **데이터 추출**: 텍스트에서 특정 정보 추출 (이름, 날짜, 금액 등)
3. **의사결정 기록**: 답변과 함께 근거/확신도 저장
4. **워크플로우 연결**: 다음 단계에 필요한 데이터 구조화

## Pydantic이란?

Python의 데이터 검증 라이브러리로, 타입 힌트를 사용해 데이터 모델을 정의합니다.

```python
from pydantic import BaseModel

class Person(BaseModel):
    name: str      # 필수 문자열
    age: int       # 필수 정수
    email: str | None = None  # 선택 문자열
```

---

# 1. Ollama 설치 및 서버 실행

In [1]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 0s (5,052 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current u

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [2]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. Pydantic 모델 정의

**코드 설명:**

### Pydantic 모델 구조
```python
class AnswerWithJustification(BaseModel):
    '''클래스 docstring = 모델에게 전달되는 지시사항'''
    
    answer: str
    '''필드 docstring = 필드 설명 (모델이 참고)'''
    
    justification: str
    '''답변에 대한 근거'''
```

**핵심 포인트:**
- 클래스의 **docstring**은 LLM에게 전달되는 지시사항
- 각 필드의 **docstring**은 해당 필드에 무엇을 넣어야 하는지 설명
- 타입 힌트(`str`, `int`, `list` 등)로 데이터 타입 지정

In [3]:
from pydantic import BaseModel

class AnswerWithJustification(BaseModel):
    '''사용자의 질문에 대한 답변과 그에 대한 근거(justification)를 함께 제공하세요.'''

    answer: str
    '''사용자의 질문에 대한 답변'''

    justification: str
    '''답변에 대한 근거'''

# 4. Structured Output 적용 및 실행

**코드 설명:**

### with_structured_output() 메서드
```python
structured_llm = llm.with_structured_output(AnswerWithJustification)
```
- 기존 LLM을 감싸서 Structured Output을 반환하는 새 객체 생성
- Pydantic 모델을 인자로 전달

### 실행 및 결과
```python
result = structured_llm.invoke('질문...')
```
- `result`는 **Pydantic 객체** (문자열이 아님!)
- `result.answer` - 답변 필드 접근
- `result.justification` - 근거 필드 접근
- `result.model_dump_json()` - JSON 문자열로 변환

In [6]:
from langchain_ollama import ChatOllama

# 기본 LLM 생성
llm = ChatOllama(model='llama3.2', temperature=0)

# Structured Output 적용
structured_llm = llm.with_structured_output(AnswerWithJustification)

# 실행
result = structured_llm.invoke('1 킬로그램의 벽돌과 1 킬로그램의 깃털 중 어느 쪽이 더 무겁나요?')

# 결과 출력
print("=== Pydantic 객체 필드 접근 ===")
print(f"답변: {result.answer}")
print(f"근거: {result.justification}")

print("\n=== JSON 형식 출력 ===")
print(result.model_dump_json(indent=2))

=== Pydantic 객체 필드 접근 ===
답변: 벽돌
근거: wall brick와 깃털은 두 가지 물질이지만, wall brick는 가볍고, 깃털은 무거우며, wall brick의 무게가 깃털보다 더 많습니다.

=== JSON 형식 출력 ===
{
  "answer": "벽돌",
  "justification": "wall brick와 깃털은 두 가지 물질이지만, wall brick는 가볍고, 깃털은 무거우며, wall brick의 무게가 깃털보다 더 많습니다."
}


In [8]:
# 추가 질문

print("\n=== 두 번째 질문 ===")
question2 = '"1 킬로그램"의 벽돌과 "1 킬로그램"의 깃털 중 어느 쪽이 더 무겁나요?'
result2 = structured_llm.invoke(question2)
print(f"질문: {question2}")
print(f"답변: {result2.answer}")
print(f"근거: {result2.justification}")
print(result2.model_dump_json(indent=2))

print("\n=== 세 번째 질문 ===")
question3 = '1 킬로그램의 "벽돌"과 1 킬로그램의 "깃털" 중 어느 쪽이 더 무겁나요?'
result3 = structured_llm.invoke(question3)
print(f"질문: {question3}")
print(f"답변: {result3.answer}")
print(f"근거: {result3.justification}")
print(result3.model_dump_json(indent=2))


=== 두 번째 질문 ===
질문: "1 킬로그램"의 벽돌과 "1 킬로그램"의 깃털 중 어느 쪽이 더 무겁나요?
답변: 벽돌
근거: 壁돌은 물질의 부피를 계산할 때 1kg당 1m^3의 부피가 있는 반면 깃털은 1kg당 0.01m^3의 부피를 가집니다. therefore, 1kg의 벽돌은 100배 더 무겁습니다.
{
  "answer": "벽돌",
  "justification": "壁돌은 물질의 부피를 계산할 때 1kg당 1m^3의 부피가 있는 반면 깃털은 1kg당 0.01m^3의 부피를 가집니다. therefore, 1kg의 벽돌은 100배 더 무겁습니다."
}

=== 세 번째 질문 ===
질문: 1 킬로그램의 "벽돌"과 1 킬로그램의 "깃털" 중 어느 쪽이 더 무겁나요?
답변: 벽돌
근거: wallball은 일반적으로 가벼우며, 깃털은 가볍지 않습니다.
{
  "answer": "벽돌",
  "justification": "wallball은 일반적으로 가벼우며, 깃털은 가볍지 않습니다."
}


---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 변경 (Ollama)
from langchain_ollama import ChatOllama
llm = ChatOllama(model='llama3.2', temperature=0)
```

> `with_structured_output()` 메서드는 동일하게 사용됩니다.

## 다양한 Pydantic 모델 예시

### 감정 분석
```python
class SentimentAnalysis(BaseModel):
    '''텍스트의 감정을 분석하세요.'''
    sentiment: str  # positive, negative, neutral
    confidence: float  # 0.0 ~ 1.0
    keywords: list[str]  # 핵심 키워드
```

### 정보 추출
```python
class PersonInfo(BaseModel):
    '''텍스트에서 인물 정보를 추출하세요.'''
    name: str
    age: int | None = None
    occupation: str | None = None
```

### 의사결정
```python
class Decision(BaseModel):
    '''주어진 상황에 대한 결정을 내리세요.'''
    decision: bool  # True/False
    reason: str
    confidence_level: str  # high, medium, low
```

## 주의사항

1. **모델 호환성**: 모든 모델이 Structured Output을 지원하지는 않음
2. **복잡한 스키마**: 너무 복잡한 중첩 구조는 오류 가능성 증가
3. **temperature=0**: 일관된 구조 출력을 위해 낮은 temperature 권장
4. **필드 설명**: docstring을 자세히 작성할수록 정확도 향상